In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

from bs4 import BeautifulSoup
import requests
import re

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
print(tf.__version__)
print(keras.__version__)

2.2.0-rc2
2.3.0-tf


In [ ]:
source_1 = requests.get("https://www.poesie-francaise.fr/poemes-charles-baudelaire/")  

soup_url = BeautifulSoup(source_1.text, "html.parser")

links = []

for link in soup_url.findAll('a', attrs={'href': re.compile("^https://www.poesie-francaise.fr/charles-baudelaire/")}):
    links.append(link.get('href'))

In [ ]:
extracted_poems = []
for link in links:
    source = requests.get(link)  
    soup = BeautifulSoup(source.text, "html.parser")
    
    title = soup.find("h2", class_="titrepoeme").text.strip().replace("Titre : ", "")
    poem = soup.find("div", class_="postpoetique").find("p")
    list_changes = {"<br/>": "\n", "<p>": "", "</p>": "", "<span class=\"decalage28\"></span>": "", "<span class=\"decalage11\"></span>": "", "<span class=\"decalage12\"></span>": "", "<span class=\"decalage2\"></span>": "", "<span class=\"decalage16\"></span>": ""  }
    for i, j in list_changes.items():
        poem = str(poem).replace(i, j)
    extracted_poems.append(title)
    extracted_poems.append(poem)

In [ ]:
with open('baudelaire.txt', 'w') as f:
    for line in extracted_poems:
        f.write("%s\n" % line)

In [ ]:
with open("baudelaire.txt") as f:
    baudelaire_text = f.read()

In [ ]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([baudelaire_text])

In [8]:
tokenizer.texts_to_sequences(["Bonjour"])

[[22, 11, 5, 27, 11, 6, 8]]

In [9]:
tokenizer.sequences_to_texts([[22, 11, 5, 27, 11, 6, 8]])

['b o n j o u r']

In [ ]:
max_id = len(tokenizer.word_index)
dataset_size = tokenizer.document_count

In [11]:
dataset_size

1

In [12]:
max_id

70

In [ ]:
[encoded] = np.array(tokenizer.texts_to_sequences([baudelaire_text])) - 1

In [14]:
encoded

array([30,  0, 14, ...,  0, 28, 13])

In [15]:
len(encoded)

235437

In [ ]:
train_size = len(encoded) * 90 // 100

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [ ]:
n_steps = 100
window_length = n_steps + 1
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [ ]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [ ]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [ ]:
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [ ]:
dataset = dataset.prefetch(1)

In [23]:
dataset

<PrefetchDataset shapes: ((None, None, 70), (None, None)), types: (tf.float32, tf.int64)>

In [ ]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id], dropout=0.2),
    keras.layers.GRU(128, return_sequences=True, dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=50)

Epoch 1/50
6619/6619 [==============================] - 94s 14ms/step - loss: 1.8474
Epoch 2/50
6619/6619 [==============================] - 94s 14ms/step - loss: 1.6467
Epoch 3/50
6619/6619 [==============================] - 94s 14ms/step - loss: 1.5905
Epoch 4/50
6619/6619 [==============================] - 94s 14ms/step - loss: 1.5619
Epoch 5/50
6619/6619 [==============================] - 95s 14ms/step - loss: 1.5423
Epoch 6/50
6619/6619 [==============================] - 94s 14ms/step - loss: 1.5274
Epoch 7/50
6619/6619 [==============================] - 94s 14ms/step - loss: 1.5173
Epoch 8/50
6619/6619 [==============================] - 94s 14ms/step - loss: 1.5088
Epoch 9/50
6619/6619 [==============================] - 94s 14ms/step - loss: 1.5025
Epoch 10/50
6619/6619 [==============================] - 94s 14ms/step - loss: 1.4964
Epoch 11/50
6619/6619 [==============================] - 94s 14ms/step - loss: 1.4920
Epoch 12/50
6619/6619 [==============================] - 94s 14

In [ ]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [26]:
X_new = preprocess(["Bonjour comment all"])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1]

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


'e'

In [ ]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [ ]:
def complete_text(text, n_chars=250, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [29]:
print(complete_text("t", temperature=0.2))

KeyboardInterrupt: ignored

In [ ]:
print(complete_text("l", temperature=1))

In [ ]:
print(complete_text("l", temperature=2))

In [ ]:
print(complete_text("u", n_chars=200, temperature=0.5))

In [ ]:
print(complete_text("l", temperature=0.4, n_chars=100))

In [ ]:
dataset_1 = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset_1 = dataset_1.window(window_length, shift=n_steps, drop_remainder=True)
dataset_1 = dataset_1.flat_map(lambda window: window.batch(window_length))
dataset_1 = dataset_1.batch(1)
dataset_1 = dataset_1.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset_1 = dataset_1.map(
lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset_1 = dataset_1.prefetch(1)

In [ ]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True, dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
])

In [ ]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [61]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset_1, epochs=50, callbacks=[ResetStatesCallback()])

Epoch 1/50


InvalidArgumentError: ignored